In [ ]:
# spyder에서 돌린 코드 코랩에서 안돌아가요...


# 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from urllib.parse import quote
from webdriver_manager.chrome import ChromeDriverManager

import warnings
warnings.filterwarnings('ignore')


# 기본 세팅
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

df = pd.read_csv('noncontain.csv')
df2 = pd.DataFrame(columns = ['movie_name', 'ost_name', 'artist_name', 'year'])


# 네이버 영화 링크 가져오기
url = "https://movie.naver.com/"
driver.get(url)
driver.implicitly_wait(3)
time.sleep(1.5)


for i in range(100, len(df['movie_name'])):
    try:
        if df['장르'][i] == '0':
            print('검색 시작')
            m_n = df['movie_name'][i]
            year = df['year'][i]
            ost_n = df['ost_name'][i]
            art_n = df['artist_name'][i]
            
            # 검색창에 단어 검색하기
            input_b = driver.find_element(By.XPATH, '//*[@id="ipt_tx_srch"]')
            time.sleep(1)
            input_b.send_keys(m_n)
            time.sleep(1)
            a = driver.find_element(By.XPATH, '//*[@id="jSearchArea"]/div/button')
            driver.execute_script("arguments[0].click();", a)
            time.sleep(2)

            # 단어 검색후 브라우저 창 정보 가져오기    
            driver.get(driver.current_url)
            driver.implicitly_wait(3)
            time.sleep(1)
                
            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            time.sleep(2)

            # 영화정보가 담긴 li 테그 타게팅
            div = soup.select('div#content')[0]
            ul = div.select('ul.search_list_1')[0]
            li = ul.select('li')
            time.sleep(1)

            # 검색된 영화정보 5개 가져오기
            for li_n in range(0, 5):
                n_mov_n = li[li_n].select('a')[1].text.strip()  # 영화이름
                n_mov_md = li[li_n].select('dd.etc')[1].select('a')[0].text.strip() # 영화감독
                n_mov_y_ge = li[li_n].select('dd.etc')[0].text  # 제작년도+영화장르
                n_mov_y = n_mov_y_ge.split('|')[-1].strip() # 제작년도
                n_mov_ge = n_mov_y_ge.split('|')[0].strip() # 영화장르
                time.sleep(2)

                if m_n in n_mov_n and abs(int(n_mov_y)-int(year)) <=2:
                    df['장르'][i] = n_mov_ge
                    df['감독'][i] = n_mov_md
                    print(n_mov_n + ' / '  + n_mov_y+ ' / '  + n_mov_md + ' / '  + n_mov_ge)
                    df2 = df2.append({'movie_name' : m_n, 'ost_name' : ost_n, 'artist_name' : art_n, 'year' : year}, ignore_index=True)
                    break
                    
            time.sleep(1)
            
        else:
            print('데이터 있다')

    except:
        print('없다네')
        print(m_n)
        pass

df.to_csv('non_add.csv', encoding = 'utf-8-sig', index = False)          # 네이버 영화로 장르, 감독 정보를 크롤링하여 추가한 데이터
df2.to_csv('add_mv_data.csv', encoding = 'utf-8-sig', index = False)     # 네이버 영화로 크롤링한 영화 데이터 (네이버 영화에서 크롤링 했을 때 잘못 가져왔을 경우 대비용-실재로 꽤 많았다.)

UnexpectedAlertPresentException: ignored